# **Prática**

Nesta prática meu objetivo era possibilitar ciar um usuário de um paciente que deseja fazer uma consulta, junto ao seu email, motivivo da consulta, data da consulta e o médico responsável pela consulta, fazendo as validações e serializações personalizadas necessárias.

In [ ]:
!pip install pydantic
!pip install pydantic[email]
!pip install FastAPI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00


In [ ]:
from datetime import datetime
from typing import Optional
from uuid import uuid4

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

app = FastAPI()

In [ ]:
class User(BaseModel):
    model_config = {
        "extra": "forbid", #Proibe que atributos extras sejam adicionados
    }
    __users__ = [] #O restanta continua muito parecido com os anteriores, porem agora aplicaremos isso a uma API

    nome_paciente: str = Field(description="Nome do paciente", examples=["exemplo"])

    email_paciente: EmailStr = Field(description="Email do paciente", examples=["exemplo@aleatorio.com"], frozen=True)

    consulta_medico: str = Field(description="Nome do medico da consulta")

    motivo_contulta: str = Field(description="Motivo da consulta")

    consulta_data: Optional[datetime] = Field(default_factory=datetime.now, description="Data da consulta", kw_only=True)

    id: UUID4 = Field(default_factory=uuid4, description="Unique identifier", kw_only=True)

    @field_serializer("id", when_used="json") #Irá serializar o 'id' para json
    def serialize_id(self, id: UUID4) -> str:
        return str(id)

In [ ]:
@app.get("/users", response_model=list[User]) #retorna a lista de usuarios
async def get_users() -> list[User]:
    return list(User.__users__)


@app.post("/users", response_model=User) #cria um novo usuário
async def create_user(user: User):
    User.__users__.append(user)
    return user


@app.get("/users/{user_id}", response_model=User) #retorna um usuário por meio do seu 'id'
async def get_user(user_id: UUID4) -> User | JSONResponse:
    try:
        return next((user for user in User.__users__ if user.id == user_id))
    except StopIteration:
        return JSONResponse(status_code=404, content={"message": "User not found"})

In [ ]:
def main() -> None:
    with TestClient(app) as client:
        for i in range(5):
            response = client.post(
                "/users",
                json={"nome_paciente": f"exemplo {i}",
                      "email_paciente": f"exemplo{i}@aleatorio.com",
                      "consulta_medico": f"fulano",
                      "motivo_contulta": f"dor de cabeça",},)

            assert response.status_code == 200

            assert response.json()["nome_paciente"] == f"exemplo {i}", ( #verifica o nome do paciente
                "The name of the user should be User {i}"
            )
            assert response.json()["id"], "The user should have an id" #verifica se o paciente tem um id

            user = User.model_validate(response.json()) #converte de json para objeto 'user' para validação

            assert str(user.id) == response.json()["id"], "The id should be the same" #verifica se o id do paciente é o mesmo que está no json

            assert user.consulta_data, "The signup timestamp should be set" #verifica se a data da consulta foi definida

            assert user.consulta_medico == "fulano", "The doctor for the consultation must be defined" #verifica se o medico responsável pela consulta foi definida

            assert user.motivo_contulta == "dor de cabeça", "the reason for the consultation must be defined" #verifica se o motivo da consulta foi definida

        response = client.get("/users") #obtem a lista de usuários criada anteriormente

        assert response.status_code == 200, "Response code should be 200"

        assert len(response.json()) == 5, "There should be 5 users" #verifica se foram criados somente 5 usuários

        response = client.post(
            "/users",
           json={"nome_paciente": f"exemplo 5",
           "email_paciente": f"exemplo5@aleatorio.com",
           "consulta_medico": f"fulano",
           "motivo_contulta": f"dor de cabeça",},)

        assert response.status_code == 200

        assert response.json()["nome_paciente"] == "exemplo 5", (
            "The name of the user should be User 5"
        )
        assert response.json()["id"], "The user should have an id"

        user = User.model_validate(response.json())

        assert str(user.id) == response.json()["id"], "The id should be the same"

        assert user.consulta_data, "The signup timestamp should be set"

        assert user.consulta_medico == "fulano", "The doctor for the consultation must be defined"

        assert user.motivo_contulta == "dor de cabeça", "the reason for the consultation must be defined"

        response = client.get(f"/users/{response.json()['id']}") #testa obter um paciente que foi registrado
        assert response.status_code == 200
        assert response.json()["nome_paciente"] == "exemplo 5", (
            "This should be the newly created user"
        )

        response = client.get(f"/users/{uuid4()}") #testa obter um paciente que não foi registrado
        assert response.status_code == 404
        assert response.json()["message"] == "User not found", (
            "We technically should not find this user"
        )

        response = client.post("/users", json={"nome_paciente": "User 6", "email_paciente": "wrong"}) #testa a criação de um paciente com o email errado
        assert response.status_code == 422, "The email address is should be invalid"

In [ ]:
if __name__ == "__main__":
    main()